In [23]:
# Install xgboost and compatible scikit-learn version
!pip install xgboost scikit-learn==1.2.2  # Ensure compatibility

In [24]:
# Import necessary libraries
import numpy as np
import pandas as pd
import xgboost as xgb

In [25]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [26]:
# Load dataset
file_path = "/content/output.csv"  # Update with your dataset path
try:
    df = pd.read_csv(file_path)
except FileNotFoundError:
    raise FileNotFoundError(f"Dataset not found at {file_path}. Please check the file path.")

In [27]:
# Display basic dataset information
print(df.head())
print(df.info())

    ID   Source  Severity           Start_Time             End_Time  \
0  A-1  Source2         3  2016-02-08 05:46:00  2016-02-08 11:00:00   
1  A-2  Source2         2  2016-02-08 06:07:59  2016-02-08 06:37:59   
2  A-3  Source2         2  2016-02-08 06:49:27  2016-02-08 07:19:27   
3  A-4  Source2         3  2016-02-08 07:23:34  2016-02-08 07:53:34   
4  A-5  Source2         2  2016-02-08 07:39:07  2016-02-08 08:09:07   

   Start_Lat  Start_Lng  End_Lat  End_Lng  Distance(mi)  ... Roundabout  \
0  39.865147 -84.058723      NaN      NaN          0.01  ...      False   
1  39.928059 -82.831184      NaN      NaN          0.01  ...      False   
2  39.063148 -84.032608      NaN      NaN          0.01  ...      False   
3  39.747753 -84.205582      NaN      NaN          0.01  ...      False   
4  39.627781 -84.188354      NaN      NaN          0.01  ...      False   

  Station   Stop Traffic_Calming Traffic_Signal Turning_Loop Sunrise_Sunset  \
0   False  False           False          F

In [28]:
# Preprocessing: Handle missing data
missing_threshold = 0.3  # 30% threshold for dropping columns with too many missing values
df = df[df.columns[df.isnull().mean() < missing_threshold]]

In [29]:
# Convert boolean columns to integers
bool_cols = df.select_dtypes(include='bool').columns
df[bool_cols] = df[bool_cols].astype(int)

<ipython-input-29-766ddd7cd083>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[bool_cols] = df[bool_cols].astype(int)


In [30]:
# Impute numerical and categorical data
num_cols = df.select_dtypes(include=np.number).columns
cat_cols = df.select_dtypes(include='object').columns

In [31]:
# Impute numerical columns with mean
num_imputer = SimpleImputer(strategy='mean')
df[num_cols] = num_imputer.fit_transform(df[num_cols])

<ipython-input-31-35898de64f67>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[num_cols] = num_imputer.fit_transform(df[num_cols])


In [32]:
# Impute categorical columns with the most frequent value (mode)
cat_imputer = SimpleImputer(strategy='most_frequent')
df[cat_cols] = cat_imputer.fit_transform(df[cat_cols])

In [33]:
# Apply LabelEncoder to all categorical columns
for col in cat_cols:
    df[col] = df[col].astype(str)  # Ensure string type for LabelEncoder
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

In [34]:
# Ensure the target 'Severity' is integer and shift class labels
if "Severity" not in df.columns:
    raise ValueError("The dataset does not contain a 'Severity' column.")
y = df["Severity"].astype(int)  # Convert target to integer
y = y - 1  # Shift class labels to start from 0

In [35]:
# Split into features (X) and target variable (y)
X = df.drop("Severity", axis=1)  # Features

In [36]:
# Step 1: Split into 75% train + 25% temp (validation + test)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

In [37]:
# Step 2: Divide temp set into validation (15%) and test sets (10%)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.4, random_state=42, stratify=y_temp)

In [38]:
# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [39]:
# Train XGBoost model
xgb_model = xgb.XGBClassifier(
    n_estimators=100,
    random_state=42,
    use_label_encoder=False,  # Avoid warnings related to label encoding
    eval_metric='logloss'  # Specify evaluation metric
)
xgb_model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [13:15:49] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

In [40]:
# Evaluate on validation set
y_val_pred = xgb_model.predict(X_val)
print("Accuracy on validation set:", accuracy_score(y_val, y_val_pred))
print("\nClassification Report:\n", classification_report(y_val, y_val_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_val, y_val_pred))

Accuracy on validation set: 0.9668666666666667

Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        15
           1       0.97      0.97      0.97      8253
           2       0.96      0.96      0.96      6727
           3       0.00      0.00      0.00         5

    accuracy                           0.97     15000
   macro avg       0.48      0.48      0.48     15000
weighted avg       0.97      0.97      0.97     15000


Confusion Matrix:
 [[   0   15    0    0]
 [   0 8016  237    0]
 [   0  240 6487    0]
 [   0    1    4    0]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [41]:
# Evaluate on test set
y_test_pred = xgb_model.predict(X_test)
print("Accuracy on test set:", accuracy_score(y_test, y_test_pred))
print("\nClassification Report:\n", classification_report(y_test, y_test_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_test_pred))

Accuracy on test set: 0.9637

Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           1       0.96      0.97      0.97      5503
           2       0.96      0.96      0.96      4484
           3       0.00      0.00      0.00         3

    accuracy                           0.96     10000
   macro avg       0.48      0.48      0.48     10000
weighted avg       0.96      0.96      0.96     10000


Confusion Matrix:
 [[   0   10    0    0]
 [   0 5338  165    0]
 [   0  185 4299    0]
 [   0    1    2    0]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [42]:
# Function to predict severity
def predict_severity(new_data_point, trained_model, scaler):
    """
    Predict the severity of a new data point.

    Parameters:
        new_data_point (array-like or pd.Series): The new data point.
        trained_model (object): Trained model to use for prediction.
        scaler (object): Scaler used for preprocessing.

    Returns:
        int: Predicted severity class.
    """
    new_data_point = np.array(new_data_point).reshape(1, -1)

    # Check feature consistency
    if new_data_point.shape[1] != X_train.shape[1]:
        raise ValueError(f"Expected {X_train.shape[1]} features, got {new_data_point.shape[1]}.")

    # Scale the data and predict
    scaled_data = scaler.transform(new_data_point)
    return trained_model.predict(scaled_data)[0]

In [43]:
# Test with a sample new data point
new_data_point = pd.Series(np.zeros(X.shape[1]), index=X.columns)
predicted_severity = predict_severity(new_data_point, xgb_model, scaler)
print(f"Predicted Severity: {predicted_severity}")

Predicted Severity: 1


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
